# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [1]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [5]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [6]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in _index.monthsOfInterest:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino34NoDetrend,nino4NoDetrend,eastIONoDetrend,dmi,nino34,nino4,eastIO,sam,lat1,lat2
year,,,,,,,,,,
1995,2.28502,2.44189,2.85653,2.90004,2.22618,2.69166,3.03808,3.02550,3.40336,3.63989
1996,1.98380,2.17850,1.82776,3.05633,2.32933,3.52185,3.71663,1.95863,2.50440,2.47077
1997,1.91970,2.09714,2.19129,1.77596,2.89654,1.41045,2.29716,2.45682,2.88724,2.15456
1998,1.76502,3.30321,2.64186,3.07833,3.49750,2.31967,1.67788,1.82896,2.63465,1.54948


Calculate it using the function

In [7]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)

calculatedAvs.round(5)

['eastIONoDetrend', 'lat2', 'nino4NoDetrend', 'eastIO', 'nino34NoDetrend', 'dmi', 'nino34', 'sam', 'lat1', 'nino4']


,eastIONoDetrend,lat2,nino4NoDetrend,eastIO,nino34NoDetrend,dmi,nino34,sam,lat1,nino4
year,,,,,,,,,,
1995,2.85653,3.63989,2.44189,3.03808,2.28502,2.90004,2.22618,3.02550,3.40336,2.85653
1996,1.82776,2.47077,2.17850,3.71663,1.98380,3.05633,2.32933,1.95863,2.50440,1.82776
1997,2.19129,2.15456,2.09714,2.29716,1.91970,1.77596,2.89654,2.45682,2.88724,2.19129
1998,2.64186,1.54948,3.30321,1.67788,1.76502,3.07833,3.49750,1.82896,2.63465,2.64186


In [8]:
numpy.isclose(calculatedAvs.sam, expectedAv.sam)

array([ True,  True,  True,  True])